In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from functools import reduce
from collections import Counter

In [174]:
total = (df.applymap(lambda x: True if x != '' else False) * np.arange(1, df.shape[1] +1)).max(axis=1)
local_np = (df.applymap(lambda x: len(x) > 0 and not x.endswith('*')) * np.arange(1, df.shape[1] + 1)).to_numpy().flatten()
local_avg = local_np[local_np > 0].mean()
remote_np = (df.applymap(lambda x: len(x) > 0 and not x.endswith('*')) * np.arange(1, df.shape[1] + 1)).to_numpy().flatten()
remote_avg = remote_np[remote_np > 0].mean()

local_avg, remote_avg

4.125

In [171]:
# ETL
FILEPATH = './data.csv'
def etl(filepath, strip_stars=False):
    df = pd.read_csv(filepath).fillna('')
    if strip_stars:
        return df.applymap(lambda x: x[:-1] if x.endswith('*') else x)
    return df
df = etl(FILEPATH)
df.head(20)

,0,1,2,3,4,5,6,7,8
0,Alex,Jake,Charles,Scott,Diego*,Yukie,David,,
1,Alex,Jake,Scott,David,Leila,Yukie,Charles*,Annabelle*,Diego*
2,Annabelle,Charles,Jake,David,Leila,Yukie,Scott,Diego*,Alex
3,Alex,David,Leila,Diego*,Jake,Charles,Scott,Annabelle,
4,Alex,Leila,Scott*,Charles*,Yukie*,David,Annabelle,Jake,
5,Alex,Jake,Yukie,Charles,Scott,Annabelle,David,Diego*,Leila*
6,Alex,Scott,Annabelle,Yukie,David,Charles,Jake*,,
7,Alex,Yukie,Leila*,Diego,Charles*,Annabelle,Scott,,
8,Alex,Jake,Annabelle,Yukie,Charles,Scott,Diego*,David*,Leila
9,Annabelle,Scott,David,Leila,Yukie,Jake,Charles,Diego*,


In [31]:
names_dict = {
    'al':'Alex',
    'ja':'Jake',
    'ch':'Charles',
    'sc':'Scott',
    'di':'Diego',
    'yu':'Yukie',
    'da':'David',
    'li':'Leila',
    'an':'Annabelle'
}


True

# CHECK TO SEE IF BEING REMOTE CHANGED LIKELIHOOD OF BEING PICKED EARLY

In [161]:
total = (df.applymap(lambda x: True if x != '' else False) * np.arange(1, df.shape[1] +1)).max(axis=1)
local_df = (df.applymap(lambda x: x != '' and '*' not in x))
local_avg = ((local_df * np.arange(1, df.shape[1] +1)).sum(axis=1) / local_df.sum(axis=1)).mean()
remote_df = (df.applymap(lambda x: '*' in x))
remote_avg = ((remote_df * np.arange(1, df.shape[1] +1)).sum(axis=1) / remote_df.sum(axis=1)).mean()
office_avg = total.mean()
print('Average number of people in standup: {:.1f}'.format(office_avg))
print('If you are local, your average turn number is: {:.1f}'.format(local_avg))
print('If you are remote, your average turn number is: {:.1f}'.format(remote_avg))
print('If you are remote, you will be picked {:.1f} people later than if you work in the office.'.format(remote_avg - local_avg))
print('AKA if you are remote, you will be picked {:.1f}% later than if you work in the office.'.format(100 * (remote_avg - local_avg) / office_avg))

Average number of people in standup: 7.8
If you are local, your average turn number is: 4.2
If you are remote, your average turn number is: 5.6
If you are remote, you will be picked 1.4 people later than if you work in the office.
AKA if you are remote, you will be picked 18.4% later than if you work in the office.


0     6
1     8
2     8
3     7
4     7
5     8
6     6
7     6
8     8
9     7
10    8
11    6
12    6
13    6
14    7
15    6
16    6
17    6
dtype: int32

In [110]:
clean_df = etl(FILEPATH, strip_stars=True)
names = set(clean_df.to_numpy().flatten())
names.remove('')
names

{'Alex',
 'Annabelle',
 'Charles',
 'David',
 'Diego',
 'Jake',
 'Leila',
 'Scott',
 'Yukie'}

In [111]:
def get_vals(name:str, df):
    data = (clean_df == name ) * np.arange(df.shape[1])
    avg = data.sum(axis=1)
    avg = avg.where(lambda x: x > 0)
    avg.dropna(inplace=True)
    mean = avg.mean()
    mean = '{:.2f}'.format(round(mean, 2))
    return mean

In [112]:
popular = {k:get_vals(k, clean_df) for k in names}
dict(sorted(popular.items(), key=lambda item: item[1]))

{'Annabelle': '3.50',
 'Jake': '3.59',
 'Leila': '3.60',
 'Scott': '3.79',
 'Charles': '3.80',
 'David': '3.92',
 'Yukie': '4.24',
 'Diego': '4.75',
 'Alex': '8.00'}

## Number of times people were at standup

In [144]:
def get_counts(data:pd.DataFrame):
    li = df.applymap(lambda x: x[:-1] if x.endswith('*') else x).to_numpy().flatten()
    li = filter(lambda x: x != '', li)
    c = Counter(li)
    return dict(sorted(c.items(), key=lambda item: item[1]))
c = get_counts(df)
pd.DataFrame(data=c.items()).set_index(0)

,1
0,
Alex,12
Scott,14
David,15
Leila,15
Diego,16
Annabelle,16
Jake,17
Yukie,17
Charles,18


## Number of times everyone called in remotely

In [53]:
def get_wfh_counts(data:pd.DataFrame):
    df = data.copy()
    li = filter(lambda x: '*' in x,df.to_numpy().flatten())
    li = map(lambda x: x[:-1], li)
    c = Counter(li)
    return dict(sorted(c.items(), key=lambda item: item[1]))
get_wfh_counts(df)

{'David': 1,
 'Yukie': 2,
 'Jake': 2,
 'Leila': 3,
 'Charles': 4,
 'Scott': 4,
 'Annabelle': 6,
 'Diego': 14}

In [114]:
def get_previous_name_counts(data:pd.DataFrame, name:str):
    c = Counter()
    for li in data.to_numpy():
        indicies = filter(lambda x: li[x + 1] == name, range(len(li)-1))
        names = [li[i] for i in indicies]
        c.update(Counter(names))
    f = {k:v for k,v in c.items() if v == c.most_common()[0][1]}
    return list(f.keys()), c.most_common()[0][1]
get_previous_name_counts(clean_df, 'Yukie')

(['Diego', 'Leila'], 4)

# And now the best part: everyone's favorite people!
## This section says who picked you the most!

In [126]:
favorite_persons = {k:get_previous_name_counts(clean_df, k) for k in names}
favorite_persons = dict(sorted(favorite_persons.items(), key=lambda item: item[0], reverse=False))
for k,v in favorite_persons.items():
    suffix = 's!' if v[1] > 1 else '!'
    print(f'{k} was picked by {" and ".join(v[0])} {v[1]} time' + suffix)

Alex was picked by Diego 1 time!
Annabelle was picked by Charles 5 times!
Charles was picked by Jake and Yukie 4 times!
David was picked by Yukie 4 times!
Diego was picked by Scott and Annabelle and Leila 4 times!
Jake was picked by Alex 4 times!
Leila was picked by David 8 times!
Scott was picked by Charles 4 times!
Yukie was picked by Diego and Leila 4 times!


In [147]:
def get_average_num(df:pd.DataFrame):
    li = []
    for row in df.to_numpy():
        n = len(list(filter(lambda x: len(x) > 0, row)))
        li.append(n)
    return round(np.array(li).mean(),2)

In [148]:
get_average_num(df)

7.78